# Output Fixing Parser

- 著者: [Jeongeun Lim](https://www.linkedin.com/in/jeongeun-lim-808978188/)
- ピアレビュー: [Junseong Kim](https://www.linkedin.com/in/%EC%A4%80%EC%84%B1-%EA%B9%80-591b351b2/)
- 校正: [Two-Jay](https://github.com/Two-Jay)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/08-OutputFixingParser.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/08-OutputFixingParser.ipynb)
## 概要

LangChainの```OutputFixingParser```は、出力解析プロセス中に発生する可能性のあるエラーを自動的に修正するメカニズムを提供します。このパーサーは、```PydanticOutputParser```などの別のパーサーをラップするように設計されており、基礎となるパーサーが不正な形式または予期される形式に準拠しない出力に遭遇したときに介入します。これは、追加のLLM呼び出しを活用してエラーを修正し、適切なフォーマットを確保することで実現されます。

その核心において、```OutputFixingParser```は、初期出力が事前定義されたスキーマに準拠していない状況に対処します。このような問題が発生した場合、パーサーは自動的にフォーマットエラーを検出し、問題を修正するための具体的な指示を含む新しいリクエストをモデルに送信します。これらの指示は問題領域を強調し、正しい形式でデータを再構築するための明確なガイドラインを提供します。

この機能は、スキーマへの厳密な準拠が重要なシナリオで特に役立ちます。たとえば、```PydanticOutputParser```を使用して特定のデータスキーマに準拠した出力を生成する場合、フィールドの欠落や不正なデータ型などの問題が発生する可能性があります。

```OutputFixingParser```は以下のように介入します:

1. **エラー検出**: 出力がスキーマ要件を満たしていないことを認識します。
2. **エラー修正**: 問題に対処するための明示的な指示を含むフォローアップリクエストをLLMに生成します。
3. **具体的な指示による再フォーマット**: ```OutputFixingParser```は、修正指示がフィールドの欠落や不正なデータ型などのエラーを正確に識別することを保証します。指示は、LLMがスキーマ要件を正確に満たすように出力を再フォーマットするようにガイドします。

----
**実用例:**

```name```(文字列)、```age```(整数)、```email```(文字列)などの特定のフィールドを必要とするスキーマを強制するために```PydanticOutputParser```を使用しているとします。LLMが```age```フィールドが欠落しているか、```email```フィールドが有効な文字列でない出力を生成した場合、```OutputFixingParser```が自動的に介入します。次のような詳細な指示を含む新しいリクエストをLLMに発行します:

- 「出力に```age```フィールドがありません。```age```に適切な整数値を追加してください。」
- 「```email```フィールドに無効な形式が含まれています。有効なメール文字列に一致するように修正してください。」

この反復プロセスにより、最終的な出力は手動介入なしで指定されたスキーマに準拠することが保証されます。

----
**主な利点:**

- **エラー回復**: ユーザー入力を必要とせずに、不正な形式の出力を自動的に処理します。
- **精度の向上**: 出力が事前定義されたスキーマに準拠することを保証し、不整合のリスクを軽減します。
- **ワークフローの合理化**: 手動修正の必要性を最小限に抑え、時間を節約し、効率を向上させます。

### 目次

- [概要](#概要)
- [環境設定](#環境設定)
- [データモデルの定義とPydanticOutputParserの設定](#データモデルの定義とPydanticOutputParserの設定)
- [OutputFixingParserを使用して不正なフォーマットを修正](#OutputFixingParserを使用して不正なフォーマットを修正)

### 参考文献

- [LangChain APIリファレンス](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.fix.OutputFixingParser.html)
- [Pydanticドキュメント](https://docs.pydantic.dev/latest/api/base_model/)

## 環境設定

環境をセットアップします。詳細については[環境設定](https://wikidocs.net/257836)を参照してください。

**[注意]**
- ```langchain-opentutorial```は、チュートリアル用の使いやすい環境設定、便利な関数、ユーティリティを提供するパッケージです。
- 詳細については[```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi)をご確認ください。

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# 必要なパッケージをインストール
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# 環境変数を設定
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "08-OutputFixingParser",
    }
)

環境変数が正常に設定されました。


あるいは、```.env```ファイルに```OPENAI_API_KEY```を設定して読み込むこともできます。

[注意] 前の手順で```OPENAI_API_KEY```を既に設定している場合、これは不要です。

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## データモデルの定義とPydanticOutputParserの設定

- ActorクラスはPydanticモデルを使用して定義され、nameとfilm_namesは俳優の名前と出演した映画のリストを表すフィールドです。
- ```PydanticOutputParser```は、出力をActorオブジェクトに解析するために使用されます。

In [5]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


# Pydanticを使用してActorクラスを定義
class Actor(BaseModel):
    name: str = Field(description="俳優の名前")
    film_names: List[str] = Field(description="出演した映画名のリスト")


# ランダムな俳優のフィルモグラフィーを生成するクエリ
actor_query = "ランダムな俳優のフィルモグラフィーを生成してください。"

# PydanticOutputParserを使用して出力をActorオブジェクトに解析
parser = PydanticOutputParser(pydantic_object=Actor)

### 不正な形式の入力データの解析を試みる

- misformatted変数には、予期される構造と一致しない不正な形式の文字列が含まれています（"の代わりに'を使用）。
- parser.parse()を呼び出すと、形式の不一致のためにエラーが発生します。

In [6]:
# 意図的に不正な形式のデータを入力
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

try:
    parser.parse(misformatted)
except Exception as e:
    print(f"エラーが発生しました: {e}")

エラーが発生しました: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}


## OutputFixingParserを使用して不正なフォーマットを修正
### エラーを自動的に修正するためにOutputFixingParserを設定
- ```OutputFixingParser```は既存の```PydanticOutputParser```をラップし、LLMへの追加呼び出しを行ってエラーを自動的に修正します。
- from_llm()メソッドは、```OutputFixingParser```を```ChatOpenAI```に接続して、出力のフォーマット問題を修正します。

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import OutputFixingParser

# 修正指示を提供するカスタムプロンプトを定義
fixing_prompt = PromptTemplate(
    template=(
        "次のJSONは不正な形式または不完全です: {completion}\n"
    ),
    input_variables=[
        "completion",
    ],
)

# OutputFixingParserを使用してエラーを自動的に修正
new_parser = OutputFixingParser.from_llm(
    parser=parser, llm=ChatOpenAI(model="gpt-4o"), prompt=fixing_prompt
)

In [8]:
# 不正な形式の出力データ
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

### OutputFixingParserを使用して不正な形式の出力を解析
- new_parser.parse()メソッドは、不正な形式のデータを解析するために使用されます。OutputFixingParserはデータ内のエラーを修正し、有効なActorオブジェクトを生成します。

In [9]:
# 例外処理を使用して不正な形式のJSONを解析
try:
    actor = new_parser.parse(misformatted)
    print("解析されたactor:", actor)
except Exception as e:
    print("解析中のエラー:", e)

解析されたactor: Actor(name='Tom Hanks', film_names=['Forrest Gump'])


### 解析された結果を確認
- 解析後、結果は修正された形式の有効なActorオブジェクトです。初期の不正な形式の文字列のエラーは、OutputFixingParserによって自動的に修正されました。

In [10]:
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])